# Analyze Track

Load a single track into memory to perform basic analysis.

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://github.com/Logiqx/gps-wizard) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

## Notes

Loads a single GT-31 track into memory and performs some basic analysis

Useful link on [Stack Overflow](https://stackoverflow.com/questions/27083051/matplotlib-xticks-not-lining-up-with-histogram)

In [9]:
import os
import sys
import math

from datetime import datetime

import numpy as np

corePath = os.path.join('..', 'core')
if corePath not in sys.path:
    sys.path.extend([corePath])

from file_reader import getFileReader

## Main Functions

In [10]:
def loadTrack(filename):
    '''Analyze a GPS track'''

    fileReader = getFileReader(filename)
    if os.path.splitext(filename)[1].lower() == '.ubx':
        fileReader.load(ignoreChecksums=True)
    else:
        fileReader.load()

    track = fileReader.tracks[0]
    
    return track

In [11]:
def summariseField(track, fieldName):
    '''Summarise field'''
    
    minVal = track.data[fieldName].min()
    medVal = np.median(track.data[fieldName])
    maxVal = track.data[fieldName].max()

    print('{}: min = {:0.1f}, med = {:0.1f}, max = {:0.1f}'.format(fieldName, minVal, medVal, maxVal))


def summariseDateTimeField(track, fieldName):
    '''Summarise field'''
    
    minVal = datetime.fromtimestamp(track.data[fieldName].min())
    medVal = datetime.fromtimestamp(np.median(track.data[fieldName]))
    maxVal = datetime.fromtimestamp(track.data[fieldName].max())

    print('{}: min = {}, med = {}, max = {}'.format(fieldName, minVal, medVal, maxVal))


def summarizeTrack(filename, track):
    '''Summarise track'''

    print(filename)
    
    print('trackpoints: {}'.format(len(track.data['ts'])))
    summariseDateTimeField(track, 'ts')
    summariseField(track, 'sat')
    summariseField(track, 'hdop')
    summariseField(track, 'cog')

    print()


def checkLogging(filename, track):
    '''Summarise track'''

    tsDiff = track.data['ts'][1:] * 1000 - track.data['ts'][:-1] * 1000
    bins, counts = np.unique(tsDiff, return_counts=True)
    
    dirname = os.path.basename(os.path.dirname(filename))
    basename = os.path.basename(filename)
    
    if len(bins) > 1:
        print('\n{}/{} - {} {}'.format(dirname, basename, bins, counts))

In [12]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    #filename = os.path.join(projdir, 'sessions', '20220410', 'GW52_1Hz_K888_155800017_20220410_230017.sbp')
    #track = loadTrack(filename)
    #summarizeTrack(filename, track)

    #filename = os.path.join(projdir, 'sessions', '20220410', 'Motion_Mini_10Hz_0470_2022-04-10-1345.oao')
    #track = loadTrack(filename)
    #summarizeTrack(filename, track)

    #filename = os.path.join(projdir, 'sessions', '20220714', '0470_2022-07-14-1506.gpx')
    #track = loadTrack(filename)
    #summarizeTrack(filename, track)

    #filename = os.path.join(projdir, 'sessions', '20220714', '0470_2022-07-14-1506.oao')
    #track = loadTrack(filename)
    #summarizeTrack(filename, track)

    #filename = os.path.join(projdir, 'sessions', 'manfred', '2307_2021-09-30-1002.oao')
    #track = loadTrack(filename)
    #summarizeTrack(filename, track)

    #filename = os.path.join(projdir, 'sessions', 'manfred', '093154.UBx')
    #track = loadTrack(filename)
    #summarizeTrack(filename, track)

    #filename = os.path.join(projdir, 'sessions', '20220926', 'Speedsurfing20220926170148.fit')
    #track = loadTrack(filename)
    #summarizeTrack(filename, track)

    #for i in [619, 621, 633, 634]:
        #filename = os.path.join(projdir, 'sessions', '20220928', '{:04d}_2022-09-28-1353.oao'.format(i))
        #track = loadTrack(filename)
        #summarizeTrack(filename, track)
    
    print('Checking WSW logging:')

    #data = os.walk(os.path.join(projdir, 'sessions', 'wsw'))
    data = os.walk(os.path.join(projdir, 'sessions'))

    for root, subDirs, files in sorted(data, key=lambda x: x[0]):
        for file in sorted(files):
            ext = os.path.splitext(file)[1].lower()
            
            if ext and ext in ['.oao']:
                print('.', end='')

                filename = os.path.join(root, file)
                track = loadTrack(filename)
                checkLogging(filename, track)

    print('\nAll done!')

Checking WSW logging:
.
20211020-esp/BoomL_83AF2466E48_004.oao - [ 99. 100. 101. 200.] [    1 55771     1     5]
.
20211020-esp/Head_L_7C9EBDFAF5C8_002.oao - [ 99. 100. 101.] [    2 56050     2]
..
20211230-esp/BoomL_83AF2466E48_008.oao - [100. 200.] [43614     2]
.
20211230-esp/Boom_R_84CCA86023F8_004.oao - [-1900.    99.   100.   101.] [    1     1 43356     1]
.
20211230-esp/Head_L_7C9EBDFAF5C8_007.oao - [ 99. 100. 101.] [    1 43441     1]
.
20211230-esp/Head_R_7C9EBDFAF67C_006.oao - [-1900.    99.   100.   101.] [    1     1 43431     1]
.....
20220716/0470_2022-07-16-1202.oao - [ 200. 2200. 2400.] [25272     2     3]
.................................
oao/0470_2022-07-16-1202.oao - [ 200. 2200. 2400.] [25272     2     3]
.
oao/0470_2022-07-19-1751.oao - [ 200. 2200. 2400.] [39679     3    14]
.
oao/0470_2022-07-24-1106.oao - [ 200. 2200. 2400.] [71752    12    20]
.
oao/0470_2022-07-24-1510.oao - [ 200. 2200. 2400.] [39518     1     2]
.
oao/0470_2022-07-25-1241.oao - [ 200. 2400.